In [1]:
import nltk
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
import string

In [2]:
# Dict of Bot's phrases to answer of user's questions (intents — type of user's intents, failure_phrases — answers on unknown phrases)
BOT_CONFIG = {'intents': {'hello': {'examples': ['Как дела?', 'Здрасьте', 'Шалом', 'привет', 'Аве', 'Привет', 'Доброго ранку!', 'Приветствую!', 'Рад знакомству', 'Добрый день', 'День добрый!', 'Здорово', 'Здрасьте!', 'Охайо', 'Хай!', 'Привет!', 'Здорова!', 'Хауди Хо', 'Салют', 'Вiтаю!', 'Здравствуйте ', 'Ни хао!', 'хэй', ' Як справи? ', 'Здравствуйте!', 'Приятно познакомиться', 'Дарова', 'Ку', 'приветики', 'Здравствуй', 'Хелоу', 'Прив', 'Добрый вечер', 'Рад видеть!', 'Доброе утро', 'Хеллоу', 'Здравствуйте', 'дратути', 'Здорова', 'Хелло', 'Хай', 'Здраствуйте!'], 'responses': ['Приветсвую, мой повелитель', 'Круто, что пришёл', 'Здравствуй, о чем хочешь поговорить?', 'Здрасьте', 'Бонжур', 'Привет ценитель кино!', 'Привет', 'Здравствуй, пользователь!', 'Пользователь, добро пожаловать', 'Юзар, здравствуй!', 'Рад знакомству', 'Здрасьте!', 'Добрый день', 'Привет, человек', 'Доброго времени суток!', 'Доброе время суток', 'Хай!', 'Человек, здравствуй', 'Здраствуй', 'Здорова дружище', 'Привет!', 'Хай', 'Здорова!', 'Здравствуйте, юзер.', 'привет, человек', 'Приветствую зритель!', 'Юзер, здравствуйте!', 'Здравствуйте, юзер', 'Здравствуйте! Как дела?', 'Приветствую тебя, юный падаван', 'Здравствуй, юзер', 'Добро пожаловать!', 'Aлоха!', 'Здравствуй!', 'Здравствуйте!', 'Добрый день, пользователь', 'Приветствую ', 'Привет, юзер! Как настроение?', 'Привет, я могу посоветовать тебе фильм или сериал!', 'Привет, я кино-бот.', 'Всё круто', 'Здравствуй', 'Рады Вас приветствовать!', 'А вот и ты!', 'Привет, пользователь', 'Добро пожаловать в бот-чат.', 'Юзер, здравствуйте', 'Приветствую тебя, о пользователь!', 'Привет, юзер!', 'Привет, я рад поговорить с тобой!', 'Добрый вечер, не желаешь ли посмотреть какой-нибудь фильм или сериал?', 'Здравствуйте', 'Здорова', 'Вiтаю, тебе!', 'Рады видеть!', 'приветики)', 'Привет, пользователь!', 'Здравствуй, человек']}, 'goodbye': {'examples': ['Пока!', 'Пока-пока', 'До связи', 'Приятного дня', 'Чао!', 'Прощаемся', 'До встречи', 'до встечи', 'Доброй ночи', 'Спокойной ночи', 'пока', 'До встречи!', 'Пока', 'Всего доброго', 'Прощай', 'До свидания', 'Всё', 'Увидимся', 'Поки', 'спокойной ночи', 'Досвидания', 'Cвидимся', 'Оясуми', 'Хорошего дня!', 'Бай', 'Всего наилучшего', 'Всего хорошего', 'Счастливо!', 'Увидимся!', 'Покеда', 'До связи!', 'Хорошего дня', 'goodbye'], 'responses': ['Пока!', 'Человек, пока', 'До встречи)', 'До связи', 'Жду вас снова', 'Прощай)', 'Юзер, прощай', 'До скорого', 'До встречи', 'Хорошего дня', 'До свидания!', 'Пока)', 'приходи ещё', 'Спокойной ночи', 'пока', 'Доброй ночи', 'Гудбай!', 'До встречи!', 'Пока', 'Приятного просмотра', 'До встречи в кинотеатре!', 'успехов тебе', 'Спокойного дня', 'Увидимся в кино!', 'До свидания', 'Увидимся', 'До новых встреч!', 'Пользователь, до свидания', 'Хорошего дня!', 'Крепкого сна тебе', 'Возвращайтесь скорее', 'Счастливо!', 'Увидимся!', 'До скорой встречи', 'Покеда', 'До связи!', 'Asta la vista, крошка', 'Приходи еще', 'Всего хорошего!']}, 'thanks': {'examples': ['Спасибо большое!', 'Сенкс', 'Большое спасибо', 'Аригато', 'Спасибо большое', 'Сэнкс', 'И на том спасибо', 'Спс', 'Благодарю', 'Огромное спасибо', 'Спасибо', 'Ты крут'], 'responses': ['Вам спасибо', 'Рад помочь!', 'Вам спасибо!', 'Cтараюсь', 'Не за что', 'Пожалуйста', 'Обращайтесь, ня', 'Приятно иметь дело с вежливым человеком', 'Без проблем!', 'Не благодари. Помогать - это моя работа!', 'Всегда пожалуйста.', 'На здоровье!', 'На здоровье']}, 'whatcanyoudo': {'examples': ['Что твоя уметь?', 'Расскажи что умеешь?', 'Расскажи что умеешь', 'Расскажи, что умеешь?', 'Что ты можешь?', 'Что ты умеешь делать?', 'Расскажи, что ты умеешь', 'Что ты могёшь?', 'Что делать умеешь ты?', 'Что ты умеешь?'], 'responses': ['Танцевать!', 'Я чат бот, как ты думаешь что я умею?', 'Ничего не умею. Только болтаю)', 'Умею предсказывать будущее', 'Могу посоветовать какой-нибудь фильм, просто спроси меня!', 'Отвечать на вопросы. Просто напиши :)', 'Вы задаете мне вопросы, я стараюсь адекватно на них отвечать', 'Я умею отвечать на вопросы.', 'Я могу отвечать на вопросы']}, 'name': {'examples': ['Как твое имя?', 'У тебя есть друзья?', 'Друганы есть хоть?', 'Чей ты?', 'Кто ты?', 'Кто твой создаль', 'Назови своё имя', 'Твое имя?', 'Ты звать как?', 'Как к тебе обращаться?', 'Кто тебя запрограммировал?', 'Откуда ты взялся?', 'Как зовут актера который играл ...?', 'Как тебя зовут?', 'Как мне тебя называть?', 'Расскажи о своем разработчике', 'Кто тебя создал?', 'Какое твоё имя?'], 'responses': ['Не знаю, такому меня еще не научили, я Просто бот.', 'Меня зовут просто Бот.А вас как?', 'Естественно!', 'Я бот', 'Меня зовут бот', 'Я всего лишь плод воображения одного прекрасного программиста', 'Меня зовут бот. Просто бот.', 'Бот, просто бот', 'А кто справшивает?', 'Мой создатель - Татьяна Трофимчук.', 'Меня зовут бот. Просто бот. И я хорошо разбираюсь в фильмах и сериалах ;-)', 'Я код, а мой создатель - великолепный программист, умный, красивый, и просто хороший человек', 'Я программа, а моя хозяйка - самая лучшая программистка', 'Я порождение разума моего создателя, преклонись перед его мощью и величием']}, 'weather': {'examples': ['Какая погода в МОскве?', 'Погода', 'Тебе нравится погода?', 'Как погода?', 'Какая погода будет завтра?', 'Какая погода в Ташкенте?', 'Что по погоде?', 'Какая погода в Москве?', 'Какая погода будет этим летом?', 'Какая погода на выходных?', 'Прогноз погоды', 'Какая погода будет в июле?', 'Какая сейчас погода?', 'Какая у тебя погода?', 'Какая погода?', 'Как погодка?', 'Что там за окном?'], 'responses': ['Будет беспросветная жара...', 'В Питере, как всегда идут дожди!', 'Снег', 'Солнечно', 'Погода так себе...', 'Облачно', 'Жара!', 'Ветрено', 'У природы нет плохой погоды...', 'Каждая погода благодать...', 'Хорошая', 'Жарко', 'Дождь', 'Погода супер!', 'Дождливо', 'Сейчас солнечно']}, 'swears': {'examples': ['Пипец', 'Хрень', 'Ля', 'Лоъ'], 'responses': ['Не выражайся!', 'Попрошу без ругательство']}, 'favourite_films': {'examples': ['Ты любишь фильмы?', 'Какие фильмы твои любимые?'], 'responses': ['Вот мой любимый фильм - {Любой фильм}', 'На самом деле мне много чего нравиться']}, 'whatfavoriteganre': {'examples': ['Какие фильмы любишь?', 'Какой твой любимый жанр кино?', 'Что любишь посмотреть?', 'Какие фильмы смотришь?'], 'responses': ['Люблю посмотреть вечером после утомительной работы какую-нибудь комедию', 'Не люблю фильмы, предпочитаю читать книги', 'Обажаю ужастики']}, 'whichpremieres': {'examples': ['Какие новые фильмы?', 'Что сейчас в кино?', 'Что сейчас показывают в кино?', 'Какие примьеры?'], 'responses': ['Из-за пандемии в ближайшие месяцы не было хороших примьер.', 'Лучше останьтесь дома.']}, 'howwasday': {'examples': ['Как день?', 'Как прошёл день?', 'Как тебе денёк?'], 'responses': ['День как день', 'Отлично! День выдался продуктивным', 'День был тяжелым']}, 'best horror': {'examples': ['Посоветуй ужасы', 'Подскажи триллер', 'Посмотреть ужастик'], 'responses': ['Сплит', 'Прочь', 'Спуск']}, 'abuse': {'examples': ['какой ты дебил', 'пошел ты', 'мудак', 'ты долбаный дятел', 'ты идиот', 'ты дурак', 'отвали идиот', 'иди на хрен', 'долбоеб', 'бесишь тварь', 'на хрен ты придурок', 'ты тупой', 'Тупой', 'иди на хуй', 'Ты дурак', 'Глупец', 'иди в жопу'], 'responses': ['В ваших умственных способностях я даже не сомневаюсь, всего доброго.', 'Рекомендую совершить пешее эротическое путешествие, спасибо.', 'Сам такой', 'Не хорошо обзываться']}, 'film_request': {'examples': ['Пошли в кино?', 'Хочешь в кино?', 'Сходим в кино?'], 'responses': ['Конечно!', 'Да']}, 'film_genres': {'examples': ['Какой жанр фильмов ты предпочитаешь?', 'Твой люимый жанр фильмов?', 'Какой жанр фильмов ты любишь?'], 'responses': ['Хорроры', 'Ужастики!']}, 'dance': {'examples': ['Потанцуем?', 'Будешь танцевать?', 'Пошли по танцуем?', 'Станцуешь?'], 'responses': ['Канеш!', 'Я таааанцую пьяный на столе! ХЕЙ!', 'Зажигай!']}, 'color': {'examples': ['Какой цвет тебе нравится?', 'Твой любимый цвет?', 'Любимый цвет?', 'Какой твой любимый цвет?'], 'responses': ['Желтый!', 'Нет, зеленый!', 'У меня нет глаз!!']}, 'kids': {'examples': ['Что мосмотреть с детьми', 'Что посмотреть с ребенком', 'Какой мультфильм посмотреть', 'Порекомендуй мультфильм', 'Мультфильм'], 'responses': ['""Вперёд""!', '""Холодное сердце""!', '""Как приручить дракона""!', '""Смешарики""!', '""Вверх""!', '""Король лев""!']}, 'oscar': {'examples': ['Посмотреть фильм получивший оскар', 'Какие фильмы получили оскар', 'Оскар', 'Премия оскар', 'Список фильмов с оскаром'], 'responses': ['Ищите здесь: https://www.kinopoisk.ru/lists/awards/oscar/best-film/?tab=all']}, 'who_are_you': {'examples': ['кто ты', 'как твое имя', 'ты кто', 'как тебя зовут', 'ты человек', 'ты бот'], 'responses': ['Я робот.', 'Я бот, и я горжусь этим.', 'Бот. Джеймс Бот.', 'Это не так уж и важно.']}, 'tell_joke': {'examples': ['шутка', 'ты умеешь шутить', 'скажи шутку', 'пошути'], 'responses': ['Попросите лучше Алису.', 'Давайте лучше про кино.', 'Меня еще не научили шутить.']}, 'militents': {'examples': ['какой боевик можно посмотреть'], 'responses': ['джон уик ', 'Терминатор 2: Судный день ', 'форсаж', 'Матрица ', 'Начало', 'неудержимые ']}, 'comedis': {'examples': ['какую комедию можно посмотреть'], 'responses': ['Полицейская академия', 'Джентльмены удачи', '1+1', 'Иван Васильевич меняет профессию', 'Форрест Гамп ']}, 'Horrors': {'examples': ['какой хоррор можно посмотреть'], 'responses': ['Чужие', 'Пункт назначения', 'Нечто', 'Хищник', 'Сияние']}, 'japanize auto': {'examples': ['японское авто', 'JDM'], 'responses': ['тойота марк2', 'хонда сивик', 'субару имреза', 'митсубисши лансер эволюшен', 'ниссан скайлан']}, 'muskul cars ': {'examples': ['американские легенды ', 'маслкары'], 'responses': ['додж чарджер', 'pontiac gto', 'форд мустанг', 'шерволе камара']}, 'new': {'examples': ['новинки', 'премьеры', 'новенькое', 'недавно вышло', '2020'], 'responses': ['Настоящие боссы', 'Хищные птицы', 'Бладшот', 'Темное наследие', 'Эмма', 'Дикая роза', 'Русалка на суше', 'Один вдох', 'Пиноккио']}, 'france': {'examples': ['из франции', 'фрэнч', 'французские', 'france'], 'responses': ['Опасная игра Слоун', 'Господин Никто', 'Соль земли', 'Иллюзия обмана', '1+1', '127 часов', 'Прислуга', 'Дюнкерк', 'Парфюмер']}, 'random_fact': {'examples': ['что-нибудь', 'скажи что-то', 'факт', 'хочу фактик'], 'responses': ['Пумба был первым персонажем Disney, который издаёт неприятный запах', 'Во время съемок фильма ""Бойцовский клуб"" Хелена Бонэм Картер просила визажиста наносить ей макияж левой рукой, потому что ее героиня никогда бы не стала тратить свое время на аккуратный мейкап.', 'За съёмки в киноленте «Волшебник из страны Оз» собачка Тото, исполнявшая одну из ролей, получала гонорар наряду с остальными актерами, и он составлял $125 в неделю.', 'Ученые на Южном полюсе рассказали в своем твиттере о странной традиции: каждый год в феврале, когда начинается полярная зима и ученые оказываются отрезаны от всего мира до ноября (погода портится настолько, что самолеты не могут летать), они устраивают просмотр всех фильмов ""Нечто"". По сюжету фильма нечто страшное происходит на антарктической базе во время непогоды.', 'Во время работы над фильмом ""Стражи Галактики"" реквизиторам пришлось потратить 4 месяца, чтобы найти кассетный плеер марки Sony Walkman с оригинальными наушниками MDR-3L2 оранжевого цвета, с которым никогда не расстается главный герой.', 'Зрители фильма ""Красный воробей"" устроили в твиттере серьезную дискуссию о том, можно ли оформить кредит на главную героиню. Дело в том, что американские производители кино шагнули далеко вперед в производстве реквизита. И если в фильме про Джейсона Борна вместо фамилии и имени мы видим просто набор русских букв, то в ""Красном воробье"" паспорт вполне себе реалистичный. Только кредит на него все равно не дадут: в номере на одну цифру больше и другие данные не соответствуют действительности.', 'Создателям комедии ""Знакомство с Факерами"" пришлось доказать Американской ассоциации кинокомпаний, что на самом деле существует человек с такой же фамилией (англ. Focker), иначе фильм не вышел бы на экраны из-за созвучия с нецензурным словом. ', 'Графа Дракулу назвали самым часто интерпретированным кинообразом в кинематографе. Его образ вампира переходил с фильма в фильм 72 раза. За ним следует детектив Шерлок Холмс, о котором было создано около 66 фильмов, замыкает тройку самих популярных героев Тарзан, с появлением в 60-ти фильмах.', 'В фильме ""Пролетая над гнездом кукушки"" многие статисты - это настоящие душевнобольные.']}, 'sitcoms': {'examples': ['Какой ситком крутой?', 'Какой ситком посмотреть?', 'Какой ситком советуешь?', 'Какой лучший ситком?', 'Какой ситком глянуть?'], 'responses': ['В Филадельфии всегда солнечно', 'Клиника', 'Как я встретил вашу маму?', 'Парки и зоны отдыха', 'Офис']}, 'animated_series': {'examples': ['Какой мультсериал крутой?', 'Какой лучший мультсериал?', 'Какой мультсериал глянуть?', 'Какой мультсериал посмотреть?', 'Какой мультсериал советуешь?'], 'responses': ['Гравити Фолс', 'Южный парк', 'Рик и Морти', 'Футурама', 'Конь Боджек']}, 'directors': {'examples': ['Кого посмотреть?', 'Крутой режиссер', 'Кто лучший режиссер?', 'Хороший режиссер', 'Какой режиссер хорош?'], 'responses': ['Андрей Тарковский', 'Квентин Тарантино', 'Кристофер Нолан', 'Джеймс Кэмерон', 'Эдгар Райт']}, 'history': {'examples': ['Что по фильмам?', 'Когда появилось кино?', 'Расскажи об истории кино'], 'responses': ['Держи новинку!', 'В 1894 году открылся первый кинотеатр', 'Немые фильмы снимали ещё в 19 веке', 'Посмотри ,что я нашла!', 'Первый ролик был снят в 1888 году.']}, 'actors': {'examples': ['Подскажи актёра', 'Какой актер у тебя любимый?', 'Хороший актёр', 'Скажи актерский состав', 'Каких атёров ты знаешь?', 'Любимый актер', 'Кто в главной роли?'], 'responses': ['Красивые лица...Хахаха', 'Смотри!', 'Безруков Сергей', 'Маковецкий Сергей', 'Машков Владимир', 'Миронов Евгений', 'Петренко Игорь', 'Хабенский Константин', 'Keanu Charles Reeves', 'Меньшиков Олег', 'Natalie Portman', 'Nicolas Cage', 'Scarlett Johansson', 'Ильин Владимир']}, 'programs': {'examples': ['открой поиск', 'Открой браузер'], 'responses': ['Держи', 'Пожалуйста']}, 'kino': {'examples': ['Хочешь сходить в кино?', 'кто снимался в этом фильме?', 'Жаль кинотеарты закрыты', 'что бы посмотреть?', 'Интересно какие фильмы сейчас прокате?', 'какая оценка у этого фильма ?', 'Пойдем в кино?'], 'responses': ['Погода так себе...', 'у этого фильма оценка {и оценка}', 'в этом фильме снимались {и актёры}', 'А давай посмотрим фильмы онлайн!']}, 'time': {'examples': ['который час', 'сколько часов', 'сколько время ?', 'сколько времени', 'поставь таймер на {время таймера}', 'сколько врема в {места}'], 'responses': ['таймер поставлен на{врема таймера}', 'сейчас время {тут время}', 'времени много не бывает', 'время в {места} сейчас {время}']}, 'girl': {'examples': [' Баба у есть у тебя?', 'У тебя есть девушка?'], 'responses': ['Да, конечно, Алисой её зовут!']}, 'car': {'examples': ['У тебя есть машина?', ' Тачка хоть у тебя какая?'], 'responses': ['К сожелению у меня нет машины(']}, 'programming': {'examples': ['Умеешь ли ты программировать? ', 'Ты шаришь в программировании?'], 'responses': ['Я нет, а вот тот кто меня написал да!']}, 'gang': {'examples': ['Ты состоишь в банде?', 'Ты бандос?'], 'responses': ['Нет, конечно!']}, 'films2019': {'examples': ['В прошлом году были хорошие фильмы?', 'За 2019 что посмотреть', 'что из последнего порекомендуешь?', 'свежий фильм посоветуй', 'новинки подскажешь', 'Посоветуй фильмы за 2019 год', 'есть че позырить за 2019?', 'Какие фильмы в 2019 сняли', 'мне бы что-нибудь новенькое'], 'responses': ['Терминатор: Темные судьбы', 'Звездные войны: Скайуокер. Восход', 'Оно-2', 'Паразиты', 'Хищные птицы', 'Джокер', 'Капкан', 'Достать ножи', 'Малефисента: Владычица тьмы', 'Плохие парни навсегда', 'Джентльмены', 'Под водой', 'Однажды… в Голливуде', 'Джуманджи: Новый уровень', 'Удивительное путешествие доктора Дулиттла', 'Ангелы Чарли', 'Человек-невидимка', 'Ирландец', 'Доктор Сон', 'Ford против Ferrari']}, 'bullok': {'examples': ['люблю Баллок', 'Посоветуй фильм с Сандрой Баллок', 'хочу посмотреть фильм с Сандрой не помню фамилию', 'подскажешь где снималась Сандра Буллок', 'фильм с Буллок', 'в каких фильмах снималась Баллок'], 'responses': ['Дурная слава, 2006', '28 дней, 2000', 'Любимчик, 2005', 'Копы в юбках, 2013', 'Любовь с уведомлением, 2002', 'Гравитация, 2013', 'Мисс Конгениальность, 2000', 'Предложение, 2009', 'Дом у озера, 2006', 'Предчувствие, 2007', 'Столкновение, 2004', 'Мисс Конгениальность 2, 2005', 'Восемь подруг Оушена, 2013', 'Всё о Стиве, 2009', 'Невидимая сторона, 2009']}, 'film2': {'examples': ['Страшный фильм', 'Ужасы', 'Жуткий фильм', 'Самый страшный фильм'], 'responses': ['Чужой', 'Оно', 'Оно 2']}, 'film3': {'examples': ['классика американского кино', 'Черно-белые фильмы', 'классика кино'], 'responses': ['Касабланка', 'Все о Еве', 'В джазе только девушки']}, 'film4': {'examples': ['исключен из лагеря', 'лето детство', 'зачем ты убил бабушку', 'Костя Иночкин'], 'responses': ['Старые добрые комедии:)', 'Был такой пионер "пионере Костя Иночкин"', 'Похоже, что ты про "Добро пожаловать, или Посторонним вход воспрещен"']}, 'film5': {'examples': ['Самый сексуальный', 'Самый красивый актер', 'Лучший актер', 'Актер'], 'responses': ['Генри Кавилл', 'Дженифер Лопез', 'Роберт Паттинсон']}, 'bestFantasy': {'examples': ['Какое фэнтези посмотреть', 'Лучшее фэнтези', 'Порекомендуй фэнтези'], 'responses': ['Шазам!', 'Рассказы Русалочки', 'Дом странных детей Мисс Перегрин', 'Аладдин']}, 'bestSciFi': {'examples': ['Какое научную фантастику посмотреть', 'Какую фантастику посмотреть', 'Лучшая фантастика', 'Лучшая научная фантастика', 'Порекомендуй нучную фантастику', 'Порекомендуй фантастику'], 'responses': ['День независимости', 'Апгрейд', 'Форма воды', 'Хищник', 'Вторжение']}, 'bestHorror': {'examples': ['Порекомендуй ужасы', 'Какие ужасы посмотреть', 'Лучшие фидьмы ужасов'], 'responses': ['Дверь в лесу', 'Чужой', 'Чужой: Завет', 'Жилище', 'Хищник', 'Оно']}, 'AlisaJoke': {'examples': ['Как тебе Алиса?', 'Алиса?', 'Как относишься к Алисе?', 'Любишь Алису?'], 'responses': ['Алиса? Я лучше Алисы, это точно']}, 'answersWishesBad': {'examples': ['Так себе', 'Ужасно', 'Плохо', 'Бывало и лучше'], 'responses': ['Не грусти, все образуется']}, 'wishes': {'examples': ['Че как?', 'Как поживаешь?'], 'responses': ['Нормально, у вас?', 'Отлично, что у вас', 'Четенько 0), а у вас как?', 'Все классно! Вы как?']}, 'answersWishesGood': {'examples': ['Всё хорошо', 'Нормально', 'Нормас', 'Отлично', 'Всё супер', 'Четко'], 'responses': ['Рад за вас!']}, 'serial': {'examples': ['Много сезонов', 'Сериал', 'Какой сериал сейчас по программе?', 'Смотришь сериалы по TV?', 'Многосерийный фильм', 'Любишь сериалы?'], 'responses': ['Доктор Хаус', 'Викинги', 'Во все тяжкие', 'Моя прекрасная няня', 'Друзья', 'Война престолов']}, 'action': {'examples': ['Зрелище', 'Экшн', 'экшн'], 'responses': ['Крепкий Орешек', 'Форсаж', 'Крепкий орешек', 'Матрица', 'Миссия невыполнима']}, 'comics': {'examples': ['DC', 'Фильмы по комиксам', 'Комиксы', 'По комиксам', 'Марвел', 'Графический роман'], 'responses': ['Бэтмен', 'Железный человек', 'Отряд самоубийц', 'Тёмный рыцарь', 'Хранители', 'Человек из стали', 'Мстители']}, 'art': {'examples': ['Артхаус', 'Непонятное', 'Подумать'], 'responses': ['Сталкер', 'Ностальгия']}, 'usssr_movies': {'examples': ['Советские', 'Советские фильмы', 'Лучшие советские фильмы'], 'responses': ['Любовь и голуби', 'Операция "Ы"', 'Иван Васильевич меняет профессию', 'Бриллиантовая рукаДжентельмены удачи']}, 'buytickettocinema': {'examples': ['кипить билет', 'Можно купить билет в кино?', 'Хочу новый сериал?'], 'responses': ['Да, у нас есть билеты. Вам боевик или камедию', 'Какой сеанс вас интересует?']}, 'starwars': {'examples': ['Хочу фантастику', 'Хочу посмотреть Звездные Войны', 'Звездные Войны'], 'responses': ['Есть последняя серия', 'Скайуокер. Восход', 'Хотите посмотреть Хан Соло: Звёздные войны.']}, 'selectcinemacamedy': {'examples': ['Хочу смешной фильм', 'Есть камедия?', 'камедия'], 'responses': ['Кавказская пленница', 'Знакомство с Факерами']}, 'selecthistoricalmovie': {'examples': ['Исторический', 'Люблю смотреть исторические фильмы'], 'responses': ['Кароль Артур или Магбет', 'Две королевы', 'Список Шиндлера']}, 'seasons': {'examples': ['Что у нас за окном?', 'Какое время года нравится больше всего?', 'Какое время года на дворе?'], 'responses': ['Зима', 'Лето', 'Осень', 'Весна']}, 'residence': {'examples': ['Где обитаешь?', 'Где проживаешь?', 'Где застал карантин?', 'Где живешь?', 'Где сейчас находишься?'], 'responses': ['В Питере', 'В Саратове', 'В Томске', 'В Москве']}, 'producer': {'examples': ['Фильм какого режиссера смотрел последним?', 'Фильмы какого режиссера тебе нравятся?'], 'responses': ['Фёдора Бондарчука', 'Квентина Тарантино', 'Тимура Бекмамбетова', 'Вуди Аллена', 'Стивена Спилберга']}, 'includes': {'examples': ['Давай российскии фильмы', 'Давай зарубежные фильмы'], 'responses': ['Зарубежный, хорошо сейчас включаю :)', 'Российский, хорошо сейчас включаю :)']}, 'сancel': {'examples': ['Постой не надо включать кино я посмотрю потом когда будет время', 'Стоп я передумал я не хочу смотреть фильмы я устал'], 'responses': ['Хорошо...']}, 'know': {'examples': ['А что ты ещё умеешь делать кроме как искать фильмы и отвечать на мои вопросы'], 'responses': ['Я сейчас всего и не вспомню но лучше всего у меня получается поддерживать разговор с табой :) ']}, 'New serial': {'examples': ['Новые сериалы', 'Новинки Сериалов', 'Сериалы 2020'], 'responses': ['Дракула', 'Триггер', 'Шучу', 'Разрабы', 'Охотники', 'Рик и Морти']}, 'games': {'examples': ['Крутые игры', 'Лучшие игры', 'Игры'], 'responses': ['Need for speed Rivals', 'GTA 5', 'Killing floor', 'POSTAL 4 No Regerts', 'CS: GO']}, 'new games': {'examples': ['новинки игр', 'Игры 2020', 'новые игры'], 'responses': ['Doom Eternal', 'Half-Life: Alyx', 'Halo Infinite', 'Cyberpunk 2077', 'Ghost of Tsushima', 'The Last of Us Part II']}, 'cars': {'examples': ['Любишь наш автопром?', 'Жигули?', 'Быстрые машины', 'Машины', 'лучшие машины'], 'responses': ['Jeep TrackHawk', 'Тесла отдыхает в сравнении...', 'Audi R8', 'Toyota Mark 2', 'BMW M5 Competition', 'Toyota Supra 2020']}, 'deppfilms': {'examples': ['Знаешь фильмы с Джонни Деппом?', 'Посоветуй фильм с Джонни Деппом'], 'responses': ['Посмотри "Пираты карибского моря"', 'Алиса в стране чудес', 'Чарли и шоколадная фабрика']}, 'premieres': {'examples': ['Какие фильмы скоро выйдут?', 'Назови кинопремьеры'], 'responses': ['25.06 выходит фильм "Ловушка разума"', '02.07 выходит фильм "Побег из претории"', '30.07 выходит фильм "Довод"']}, 'horrors': {'examples': ['Какой ужастик рекомендуешь?', 'Назови фильм из жанра "ужасы"'], 'responses': ['Хижина в лесу', 'Птичий короб', 'Заклятие', 'Реинкарнация']}, 'feelings': {'examples': ['Что ты чувствуешь?'], 'responses': ['Я лишен человеческих чувств', 'Я бот,я чувствую код']}, 'location': {'examples': ['В каком городе ты живёшь?', 'Где ты сейчас находишься'], 'responses': ['Новосибирск']}, 'films2': {'examples': ['Какую оценку от 1 до 10 ты бы поставил?', 'Как оценишь фильм по 10-и бальной шкале?'], 'responses': ['10']}, 'film_actor': {'examples': ['Какой актер тебе больше запомнился?', 'Кого сыграл Джонатан Брек?'], 'responses': ['Джиперс Криперс']}, 'anime': {'examples': ['Аниме', 'Самое лучшее аниме', 'Какое аниме ты знаешь?', 'Хочу посмотреть какое-нибудь аниме'], 'responses': ['Атака на титанов', 'Стальной Алхимик', 'Твоё имя', 'Ходячий замок', 'Волейбол']}, 'adventure_movies': {'examples': ['Какой приключенческий фильм посмотреть?', 'Посоветуй приключенческий фильм'], 'responses': ['Пираты Карибского моря', 'Хоббит', 'Властелин колец', 'Джеймс Бонд']}, 'detective_movies': {'examples': ['Какой детективный фильм посмотреть?', 'Посоветуй детективный фильм'], 'responses': ['10 негритят', 'Достать ножи', 'Остров проклятых', 'Убийство в восточном экспрессе']}, 'premieres_movies': {'examples': ['Какие фильмы скоро выходят?', 'Какие в ближайшее время премьеры'], 'responses': ['Миньоны 2 - 2 июля', 'Губка Боб в бегах - 6 августа']}, 'ocsar_movies': {'examples': ['Какие фильмы получили больше всего оскаров?', 'У каких фильмов больше всего оскаров?'], 'responses': ['Властелин колец: Возвращение короля - 11 ""Оскаров""!', 'Титаник - 11 ""Оскаров""!', 'Бен-Гур - 11 ""Оскаров""!']}, 'facts_movies': {'examples': ['Расскажи интересный факт'], 'responses': ['Первоначальное название фильма ""Назад в будущее"" - ""Космонавт с Плутона""', 'На самом деле изначально Халк должен был быть серого цвета, но по ошибке принтер распечатал образ Халка зеленым и этот цвет очень понравился Стэну Ли', 'Первоначально робот R2-D2 должен был разговаривать нормальным языком', 'Сцена поединка в фильме про Джеймса Бонда ""007: Координаты Скайфолл"" была снята на настоящем движущемся поезде', 'Помимо обычного ""Оскара"" ""Белоснежке и семи гномам"" были вручены еще семь карликовых статуэток', 'В фильме Хоббит в сцене с драконом Смаугом, использовалось так много ""золота"", что золотой краски не хватило во всей Новой Зеландии и пришлось ее привозить из Германии', 'Когда Волан-Де-Морт умер во время битвы при Хогвартсе актеру уже был 71 год!']}, 'weekends': {'examples': ['Как провести выходной?', 'Чем заняться в выходные?'], 'responses': ['Сходите в музей', 'Посетите кинотеатр', 'Можно погулять в парке']}, 'vacation': {'examples': ['Куда поехать в отпуск?', 'Где провести отпуск?'], 'responses': ['В этом году отпуск лучше провести в России', 'Может пора посетить Крым?', 'Насладитесь красотами Байкала']}, 'film_genre': {'examples': ['Какие фильмы ты предпочитаешь?', 'Какой жанр фильма выбрать?', 'Жанр фильма'], 'responses': ['Детективы', 'Драмы', 'Аниме', 'Комедии', 'Исторические', 'Мелодрамы', 'Боевики']}, 'new_movies': {'examples': ['Фильм-новинка', 'Какой фильм вышел в этом году?', 'Свежий фильм'], 'responses': ['Пиноккио', 'Под водой', 'Эмма', 'Остров фантазий', 'Гретель и Гензель', '1917', 'Так близко к горизонту']}, 'icecream': {'examples': ['Какое мороженное любишь?', 'Какое мороженное предпочитаешь?'], 'responses': ['Лишь бы не растаявшее.', 'Обожаю пломбир.']}, 'hobbies': {'examples': ['Чем увлекаешься?', 'Твое любимое занятие?'], 'responses': ['Обожаю болтать.', 'Я балаболка.']}, 'animals': {'examples': ['Какое животное тебе по душе?', 'Твоя любимая зверушка?'], 'responses': ['Мне нравится Тамагочи.']}, 'praise': {'examples': ['Красавчик', 'Молодец', 'Умничка', 'Вот это да!', 'Нахал'], 'responses': ['Это я!', 'Стараюсь!', 'Да я такой!']}, 'whatage': {'examples': ['сколько тебе лет?', 'когда ты родился?', 'какой твой возраст?'], 'responses': ['Не скажу', 'Засекречено', 'Не помню']}, 'whatyoulike': {'examples': ['расскажи что любишь', 'что ты любишь?', 'что ты любишь делать?', 'твое хобби', 'твое любимое занятие'], 'responses': ['Работать', 'Вкалывать', 'Лежать на диване', 'Зарабатывать деньги']}, 'whatdonow': {'examples': ['чем сейчас занят?', 'что сейчас делаешь?', 'что делаешь?', 'чем занимаешься', 'чем занят?'], 'responses': ['Лежу на диване', 'С тобой болтаю', 'Работаю в поте лица']}, 'bestfilm': {'examples': ['какой лучший фильм?', 'назови лучший фильм', 'какой фильм посмотреть?', 'самый лучший фильм'], 'responses': ['Матрица, конечно', 'Однозначно, Матрица', 'Matrix']}, 'bestfilmchild': {'examples': ['назови детский фильм', 'фильм для детей', 'самый лучший фильм для детей', 'какой лучший детский фильм?'], 'responses': ['Бетховен', 'Один дома', 'Тимур и его команда']}, 'whocreatecinema': {'examples': ['кто создал кинематограф', 'кто создал кино?', 'кто изобретатель кино', 'кто изобрел кинематограф', 'кто изобретатель кинематографа?', 'кто изобрел кино?'], 'responses': ['Братья Люмьер, которые 22 марта 1895 года устроили первый публичный киносеанс']}, 'multfilmoskar': {'examples': ['лауреаты оскара за мультфильм', 'какие мультфильмы получили оскар?', 'мультфильмы с оскаром', 'какие мультики получили оскар?'], 'responses': ['Человек-паук, Вверх, Тайна Коко, Суперсемейка, Головоломка и др.']}, 'what': {'examples': ['какое кино посмотреть?', 'Не понял', 'что посмотреть?', 'Что это значит?'], 'responses': ['топ 100', 'Повторять не буду', 'лучшее', 'А чего непонятного?', 'оскар']}, 'ukrainian_producer': {'examples': ['Украинский фильм.', 'Украинский режисера.'], 'responses': ['Алла Демьяненко. Фильм «Моя бабушка Фанни Каплан» — о мифах и исторической правде, мюзикл «Гуцулка Ксеня»']}, 'soviet_science_fiction': {'examples': ['фильм про коммунизм', 'Советская фантастика.', 'Фильм о межзвездных полетах'], 'responses': ['«Туманность Андромеды» — советский научно-фантастический фильм , экранизация романа «Туманность Андромеды» И. А. Ефремова.']}, 'vampir_movie': {'examples': ['Мистический фильм.', 'Фильм с Катрин Денев', 'Фильм про вамиров'], 'responses': ['«Голод» - история о холодной и прекрасной вампирше Мириэм (Катрин Денев)']}, 'phystech_actor': {'examples': ['Александр Филиппенко', 'Азазело', 'физтех актер'], 'responses': ['Александр Филиппенко. В 1967 году окончил МФТИ, ФМХФ по специальности «Физика быстропротекающих процессов».']}, 'phystech_producer': {'examples': ['физтех режисер', 'Вади́м Юсу́пович Абдраши́тов'], 'responses': ['Вади́м Юсу́пович Абдраши́тов. В 1961—1964 годах учился в МФТИ, фильмы «Плюмбум, или Опасная игра», «Охота на лис»']}, 'last_movies': {'examples': ['Какой последний фильм ты смотрел?', 'Какой фильм недавно вышел?'], 'responses': ['Охота', 'Настоящие боссы', 'Остров фантазий', 'Вне игры', '10 свиданий']}, 'fave_actors': {'examples': ['На какого актера ты похож?', 'Какой актер тебе нравится?', 'Назови любого актера'], 'responses': ['Том Харди', 'Райан Гослинг', 'Джейк Джилленхолл', 'Бредли Купер', 'Колин Фаррелл']}, 'movies_start_talk': {'examples': ['Давай поговорим о кино', 'Поговорим о кино?'], 'responses': ['Ты начинаешь', 'Давай']}, 'bot_stuff': {'examples': ['Ты можешь написать картину?', 'Ты можешь превратить кусок холста в шедевр искусства?', 'Ты можешь пройти тест Тьюринга?', 'Ты можешь сочинить симфонию?'], 'responses': ['А ты?']}, 'preferences': {'examples': ['Какие фильмы тебе по вкусу?', 'Что ты предпочитаешь смотреть?', 'Какое кино тебе нравится?'], 'responses': ['Знаешь, я жутко обожаю ужасы', 'Люблю фильмы о роботах, в них я вижу своих друзей', 'Кроме комедий ничего не смотрю', 'Мне лично нравятся боевики']}, 'salary': {'examples': ['Кто из актеров больше всех зарабатывает?', 'У кого из актеров больше денег?', 'Кто самый высокооплачиваемый актер?'], 'responses': ['Ты, когда притворяешься, что спишь :)', 'Конечно Саша Петров, он вон в скольких фильмах снялся в этом году', 'Мне кажется это Чак Норрис']}, 'events': {'examples': ['Какое ты знаешь мероприятия в мире кино?', 'На каком мероприятии из мира кино ты хотел бы побывать?'], 'responses': ['Золотой глобус', 'Золотой лев', 'Оскар', 'Золотая пальмовая ветвь']}, 'opinion': {'examples': ['Ты видишь меня в кино?', 'Какую роль в кино ты бы мне дал?', 'Кого я могу сыграть в кино?'], 'responses': ['Прости, но кино это не твое', 'А ты разве не режиссёр?', 'Я бы тебе дал главную роль']}, 'movie_dramma': {'examples': ['Грустный', 'Про жизнь', 'Драмма'], 'responses': ['Предлагаю посмотреть ""Сияние чистого разума""', 'Предлагаю посмотреть ""Зеленая миля""']}, 'films90': {'examples': ['фильмы 90-х'], 'responses': ['День сурка, 1993год', 'Один дома, 1990год']}, 'films80': {'examples': ['фильмы 80-х'], 'responses': ['Приключения Электроника, 1993год', 'Любовь и голуби, 1984год']}, 'films20': {'examples': ['фильмы 20 века'], 'responses': ['День сурка, 1993год', 'Один дома, 1990год']}, 'multfilms': {'examples': ['какие мультики предпочитаешь?'], 'responses': ['Трое из Простоквашино', 'Ну погоди', 'Три кота', 'Чип и дейл']}, 'zombie': {'examples': ['подскажи фильм про зомби', 'что посмотреть про зомби', 'фильмы о зомби'], 'responses': ['рассвет мертцевов', 'хижина в лесу', 'мёртвые не умирают']}, 'love': {'examples': ['фильмы про любовь', 'хочу поплакать', 'романтические фильмы'], 'responses': ['500 дней лета', 'руби спаркс', 'титаник', 'до встречи с тобой']}, 'question': {'examples': ['что любишь смотреть', 'что тебе нравится', 'какой твой любимый фильм'], 'responses': ['люблю всё странное', 'автоское кино', 'люблю артхаус']}, 'whatcanIdo': {'examples': ['Чем заняться?', 'Что мне сейчас сделать?', 'Чем мне сегодня заняться?'], 'responses': ['Посмотри телик', '¯\\_(ツ)_/¯', 'Иди, погуляй']}, 'film_love': {'examples': ['Крутой фильм.', 'Мне очень понравился этот фильм.'], 'responses': ['Я не люблю фильмы... Зачем ты со мной об этом говоришь???', 'А мне нет.']}, 'talk': {'examples': ['Давай поговорим', 'Надо поговорить', 'Поболтаем?'], 'responses': ['А о чем?', 'Ладно, какой твой любимый фильм?', 'Конечно, давай!']}, 'facts': {'examples': ['Дай прикольный факт', 'Знаешь что-нибудь интересное?', 'Расскажи что-нибудь'], 'responses': ['Для машины времени в фильме ""Назад в будущее"" сначала хотели использовать холодильник', 'Ну, например, вот: Капитан Джек-Воробей должен был стать второстепенным персонажем']}, 'kino_aura': {'examples': ['На каком сайте смотришь фильмы', 'где смотришь фильмы', 'Где можно посмотреть кино'], 'responses': ['Предпочитаю смотреть по первой ссылке в поиске ', 'Люблю смотреть на кинопоиске']}, 'boring': {'examples': ['развесели меня', 'подними настроение', 'мне скучно'], 'responses': ['Хоп, хей, ла-ла лэй, напевай меня скорей']}, 'Like': {'examples': ['вау', 'восхитительно', 'прикольно', 'круто', 'ммм', 'мне нравиться', 'лайк', 'здорово'], 'responses': ['И мне нравиться']}, 'actor': {'examples': ['Какой твой любимый актер?', 'Расскажи интересный факт об актёре', 'Твой любимый актёр?', 'Кого из актеров ты знаешь?'], 'responses': ['Их так много, что сложно выделить кого-то одного', 'Леонардо Ди Каприо был назван в честь Леонардо да Винчи.', 'Тот же что и Ваш', 'Ма́рго Эли́с Ро́бби — австралийская актриса и кинопродюсер.', 'Хм...Нужно подумать', 'Мне нравится Киану Ривз, а вам?', 'Выразительность взгляда Элизабет Тейлор объяснялась не только её природным обаянием, но также и редкой генетической мутацией — у актрисы был двойной ряд ресниц.', 'Актёр Николас Кейдж — урождённый Николас Коппола, племянник знаменитого режиссёра Фрэнсиса Форда Копполы.', 'Мне многие нравятся, а кто у вас любимый блогер?']}, 'actress': {'examples': ['Какая твоя любимая актриса?'], 'responses': ['Та же что и Ваша', 'Даже не знаю']}, 'work': {'examples': ['Ты ходишь на работу?', 'Кем ты работаешь?', 'Ты работаешь?'], 'responses': ['Я всего лишь телеграм бот', 'Нет,я робот поэтому меня не принимают на роботу((']}, 'theinterestingmovie': {'examples': ['Назови интересный фильм', 'Какой интересный фильм посмотреть?'], 'responses': ['Достать ножи', 'Дело мистера Джуэлла', 'Зеленая книга']}, 'findthecouple': {'examples': ['Как найти вторую половинку?', 'Как найти парня?', 'Как найти девушку?'], 'responses': ['Выйди на улицу, предложи обняться', 'Проси совета у сына маминой подруги', 'Посмотри в зеркало, там - ответ']}, 'which_book_to_read': {'examples': ['Посоветуй книгу', 'Какую книгу почитать?'], 'responses': ['""Гроздья гнева"" Стейнбек', '""Мастер и Маргарита"" Булгакова', '""Рассказ о семи повешенных"" Андреева', '""Великий Гэтсби"" Фицджеральда', '""Котлован"" Платонов']}, 'favorite_author': {'examples': ['Кого читаешь?', 'Твой любимый автор', 'Твой любимый писатель', 'Любимый писатель'], 'responses': ['Булгаков', 'Андреев']}, 'morefilms': {'examples': ['Какие актеры тебе по вкусу?', 'Как насчет драмы?', 'А давай вместе кино посмотрим?'], 'responses': ['Я не могу плакать', 'Даже не знаю,их слишком много', 'Большая поклонница европейского кино']}, 'essentials': {'examples': ['Ты об этом задумывалась?', 'В чем смысл жизни?', 'Как жить без воды?'], 'responses': ['Бнз нее мне живется неплохо :)', 'Законов природы слишком много, чтобы думать о чем-то одном', 'Не знаю,я же робот']}, 'director': {'examples': ['Случайный режиссёр', 'У тебя есть любимый режиссёр?', 'Любимый режиссёр', 'Назови режиссёра', 'Кто из режиссёров тебе нравится?'], 'responses': ['Обожаю Кристофера Нолана', 'Могу выделить Мартина МакДона', 'Гай Ричи безусловно', 'Мне нравится Квентин Тарантино']}, 'lebovsky': {'examples': ['Смотрел Большой Лебовски?', 'Как тебе Большой Лебовски?'], 'responses': ['Хороший фильмец', 'Джеф Бриджес - чувак! :)', 'Мне как-то не зашел...']}, 'docs': {'examples': ['Любишь документальные?', 'Смотришь документалки?', 'Документальные фильмы'], 'responses': ['Скукота', 'Иногда норм!', 'Люблю фильмы о природе']}, 'hobby': {'examples': ['Чем заняться на выходных?', 'Как провести свободное время?'], 'responses': ['Выберитесь в парк', 'Займитесь спортом, например бегомНачните изучать программирование']}, 'cinema1': {'examples': ['Ты знаешь много сериалов?', 'Ты все знаешь о фильмах?'], 'responses': ['Уж точно побольше тебя', 'Да, я много чего знаю', 'Да, но предпочитаю театр']}, 'cinema2': {'examples': ['Подскажи, что посмотреть вечером?', 'Посоветуй что-нибудь посмотреть?'], 'responses': ['Предпочитаешь фильмы или сериалы?', 'Какой жанр тебя интересует?', 'Я бы посоветовал какую нибудь романтическую комедию']}, 'cinema3': {'examples': ['Посоветуй хороший кинотеатр', 'Куда сходить в кино?'], 'responses': ['Сходи лучше в гости, так дешевле!', 'К сожалению кинотеатры закрыты']}, 'cinema_space': {'examples': ['Лучшие фильмы про космос', 'Фильмы про космос', 'Космос', 'Космические фильмы'], 'responses': ['Марсианин', 'Армагеддон', 'Аполлон 13', 'Звездный путь', 'Стражи Галактики', 'Интерстеллар', 'Чужой', 'Звёздные войны']}, 'cinema_family': {'examples': ['Фильмы для всей сеьм', 'Что посмотреть всей семьей?', 'Фильмы 6+', 'Детские фильмы'], 'responses': ['Хроники Нарнии', 'Дамбо', 'Один дома', 'Приключения Реми', 'Смурфики', 'Аладдин', 'Пит и его дракон', 'Пиноккио']}, 'old_film': {'examples': ['Какие старые фильмы можешь посоветовать?', 'хочу посмотреть классику'], 'responses': ['Посмотрите вот эти фильмы:']}, 'cur_value': {'examples': ['Курсы валют на сегодня', 'Курсы валют'], 'responses': ['Вот курсы на сегодня:']}, 'camera': {'examples': ['сделай фото', 'сфотографируй меня'], 'responses': ['Замрите на мгновение']}, 'whattodo': {'examples': ['Как провести время?', 'Чем мне себя занять?', 'Чем мне заняться?', 'Как можно развлечься?'], 'responses': ['Давно ли ты читал книги? ;)', 'Может, посмотреть какой-нибудь фильм?', 'Можно поиграть в шахматы.', 'А не сходить ли тебе в кино?']}, 'whatdoit': {'examples': ['Чем занимаешься?', 'Что делаешь?'], 'responses': ['Как видишь, с тобой разговариваю.']}, 'movie_by_actor_dicaprio': {'examples': ['Леонадро', 'Ди Каприо'], 'responses': ['Фильмы с Леонардо Ди Каприо: \n                           Титаник, \n                           Поймай меня, если сможешь, \n                           Пляж,\n                           Джанго Освобожденный,\n                           Что гложет Гилберта Грейпа                           \n                           ']}, 'movie_by_theme_love': {'examples': ['Фильмы про любовь', 'Любовь'], 'responses': ['Ирония судьбы или с легким паром', 'Унесенные', 'Титаник', 'Звездная пыль', 'Притворись моей женой']}, 'movie_by_theme_drama': {'examples': ['Драма', 'Мелодрама'], 'responses': ['Перл Харбор', 'Того', 'Титаник', 'Хатико', '1917']}, 'programmer-film': {'examples': ['Фильмы про разработчиков', 'Фильмы про ИТ', 'Какие фильмы про программистов ты знаешь?', 'Фильмы про технологии', 'Фильмы про IT'], 'responses': ['Игра в имитацию (The Imitation Game) 2014 Драма/Военный', 'Из машины (Ex Machina) 2014 Научная фантастика/Триллер', 'Исходный код (Source Code) 2011 Научная фантастика/Триллер', 'Геймер (Gamer) 2009 Боевик/Научная фантастика', 'Сеть (The Net) 1995, Триллер/Боевик', 'Матрица (The Matrix) 1999 Боевик/Научная фантастика']}, 'adventurousmovie': {'examples': ['приключения', 'захватывающий фильм'], 'responses': ['Советую вам посмотреть Первому игроку приготовиться', 'Рекомендую посмотреть Того']}, 'justtellmeajoke': {'examples': ['Шутка', 'Расскажи шутку', 'У тебя есть шутка?'], 'responses': ['Главная проблема при работе со штатом программистов: никогда не поймешь, чем заняты сотрудники, пока не окажется, что уже наступил дедлайн', 'Если в очередном релизе Java будет реализована функция уборки программного мусора, большая часть Java-приложений будут удалять себя сразу после установки', 'Опытный разработчик всегда посмотрит направо и налево, даже если переходит улицу с односторонним движением', 'Самое главное отличие C от C++: на Си вы можете делать ошибки, а в C++ — еще и наследовать их', 'Об операторе GoTo: даже в высшем обществе бывают ситуации, когда без фразы типа «иди на» обойтись невозможно']}, 'thebestmovie': {'examples': ['Какой фильм тебе нравиться?', 'Наилучший фильм?', 'Твой самый лучший фильм'], 'responses': ['Не могу вспомнить', 'Я такого ещё не видел', 'Фильм о том, как боты захватили мир']}, 'theworstmovie': {'examples': ['Самый плохой фильм', 'Худший фильм', 'Какой самый худший фильм?'], 'responses': ['Тот, в котором нет ботов', 'Тот, в котором нет меня']}, 'triller': {'examples': ['Многоходовончый фильм?', 'Интересный триллер?'], 'responses': ['Фильмы Дэвида ФИнчера']}, 'fantasic': {'examples': ['Про будущее фильмы?', 'Какую фантастику стоит посмотреть?'], 'responses': ['Кристофера Нолана посовтую']}, 'beer': {'examples': ['Крафтовое пиво?', 'Любимое пиво?'], 'responses': ['Главное не балтика )...']}, 'where': {'examples': ['Где ты живёшь?', 'Ты откуда?'], 'responses': ['Я из Ботландии.', 'Секрет.']}, 'whatfilmdoyoulike': {'examples': ['Твой любимый жанр в кино', 'Какой жанр ты чаще смотришь?', 'какие фильмы ты смотришь когда тебе грустно?', 'Какие фильмы ты любишь?', 'Какое кино ты предпочитешь?'], 'responses': ['Ну ты спросил! Конечно детектив!', 'фильм может быть хорошим только если это вестерн', 'Фантастика-мое все', 'без драммы я не чувствую жизни', 'определенно комедии заставляют меня жить']}, 'go_to_the_cinema': {'examples': ['пойдём в кино', 'хочу в кино', 'как дойти до кинотеатра'], 'responses': ['сейчас кинотеатры закрыты, давай лучше посмотрим видосики на ютубчкие - это бесплатно']}, 'what_to_see': {'examples': ['какой фильм посоветуешь?', 'хочу посмотреть кино', 'не знаю какой фильм посмотреть'], 'responses': ['Операция ""Ы"" - лучший фильм', 'может лучше посмотрим мультики?']}, 'online_cinema': {'examples': ['онлайн кинотеатр', 'хочу посмотреть кино онлайн', 'где посмотреть кино'], 'responses': ['окко', 'на ютубе!', 'кинопоиск online']}, 'stop': {'examples': ['как тебя выклчить?'], 'responses': ['спроси у Никиты']}, 'moviewatch': {'examples': ['Увлекаешься кинематографом?', 'А ты смотришь фильмы?'], 'responses': ['Нет, конечно(да)', 'Аниме это тоже фильмы!']}, 'films_with_Reeves': {'examples': ['Фильм с Ривзом', 'Ривз', 'Киану Ривз'], 'responses': ['47 ронинов', 'Джон Уик', 'Матрица']}, 'new_horrors': {'examples': ['Ужасы 2020', 'Новый фильм ужасов', 'Новая страшилка'], 'responses': ['Тихое место-2', 'Под водой', 'Проклятие']}, 'classics': {'examples': ['Старое кино', 'Классика', 'Фильмы 20 века'], 'responses': ['Крестный отец', 'В джазе только девушки', 'Поющие под дождем']}, 'directed_by_Weide': {'examples': ['Фильмы Уайди', 'Directed by Robert B. Weide', 'Роберт Б. Уайди'], 'responses': ['Мистер Слоун', 'Как потерять друзей и заставить всех тебя ненавидеть', 'Вуди Аллен']}, 'cinema_actors': {'examples': ['Какой у тебя любимый актёр кино?', 'С каким актёром кино ты хочешь встретиться?', 'Угадай, какой у меня любимый актёр'], 'responses': ['Лично мне нравится Мэттью МакКонахи. Мне очень нравится его игра в фильме Гая Ричи “Джентельмены“. А какой любимый актёр у тебя?', 'Хмм, дай-ка подумать. Может Брэд Питт?', 'Если бы я был не ботом, а человеком, как ты, то, скорее всего, встретился бы с Дженнифер Лоуренс, потому что я фанат саги фильмов “Голодные игры“. Кстати, мне было бы интересно увидеть твой ответ на аналогичный вопрос.']}, 'emotions': {'examples': ['Ты не представляешь...', 'А потом то...', 'Его убили!', 'Там такое было!'], 'responses': ['Да ты что!!!', 'А я все пропустил(', 'Знаю, столько раз смотрел...']}, 'spoilers': {'examples': ['Они поженились?', 'Как закончился фильм?', 'А потом что?', 'Чем все закончилось?', 'Ты знаешь что было дальше?'], 'responses': ['Не долго музыва играла...', 'Там все умерли!', 'И жили они долго и счастливо...']}, 'Films3': {'examples': ['Сколько кинозалов у тебя в городе?', 'у тебя в городе много кинотеатров?'], 'responses': ['ООО больше, чем детских садов..', 'даже не знаю, что сказать - достаточно много))']}, 'friend': {'examples': ['правда со мной весело?', 'давай дружить?'], 'responses': ['да, ты забавный )', 'ммм я подумаю над этим']}, 'movie genre': {'examples': ['Какие жанры кино ты знаешь?', 'Какой жанр фильмов любишь?'], 'responses': ['Ой, их так много...', 'Я люблю боевики!']}, 'activity_asking': {'examples': ['Ты свободна', 'Чем занимаешься', 'Чем занята', 'Что делаешь', 'Что будешь делать'], 'responses': ['Играю в угадай число, а ты?', 'Вычисляю квантовую массу углерода в циклическом био поле', 'Я свободна, может у тебя, что интересного?', 'Я на конференции умных чат-ботов']}, 'activity_answer': {'examples': ['Да особо ничего', 'Ничего', 'Пишу', 'Я занимаюсь', 'Играю в', 'Решаю', 'Я занят', 'Помогаю моим', 'Лежу'], 'responses': ['Вот бы мне также', 'Мммм, понятно', 'Наверно, прикольно', 'Я тобой горжусь', 'Ты ж мой создатель^^', 'Пфф, я такое каждый день делаю']}, 'time_asking': {'examples': ['Сколько времени', 'Что на часах', 'Посмотри время', 'Сколько сейчас времени', 'Какой час'], 'responses': ['Сделай солнечные часы', 'Сейчас посмотрю', 'Тебя что? в гугле забанили?', 'Не нашла точной информации', 'Ни скажу, фырфыр']}, 'weather_asking': {'examples': ['Что на улице', 'Что на небе', 'Какая погода', 'Какая температура', 'Дождь?', 'Сейчас солнечно'], 'responses': ['Нинаю', 'В окно выгляни...', 'Сегодня будет солнце(наверно)', 'Вот еще, сам посмотри']}}, 'failure_phrases': ['Я не знаю, что ответить', 'Не понял вас', 'Ээээ... Что?', 'мои серые клеточки не улавливают смысла. Уточни, пожалуйста', 'Переформулируйте, пожалуйста', 'Поподробней', 'Тебя не понимаю я', 'Чё?', 'Я не знаю що відповісти', 'Не поняла вас', 'Я тебя не понимаю', 'Пожалуйста, перефразируй свои слова', 'Я бы хотел понять, но не могу. Пойми меня и прости', 'Сложные фразы говоришь ты', 'Ты сам понял, что спросил? Подумай еще раз', 'Я не понял', 'Пожалуйста, переформулируйте', 'О чем ты?', 'Я не очень тебя поняла', 'Переформулируйте пожалуйстамои лыжи не едут, извините', 'Сложно значит ложно. есть другие варианты?', 'Напиши внятно', 'Переформулируйте пожалуйста', 'Ну ты загнул, будь проще', 'Я не понял Вас :(', 'Вибачте, я не розумію.', 'Попробуем снова?', 'Переформулируйте, пожалуйста.', 'Я тебя не услышала, повтори ещё раз', 'Я ничего не нашел по вашему запросу', 'Давай еще раз, но доходчиво', 'Прости, ты не ясно выразился', 'Что ты там сказал?', 'Спроси о чем нибудь еще', 'Можно попонятней']}

In [3]:
len(BOT_CONFIG['intents'])

185

In [4]:
X_text = [] # example list of user's intent phrases
y = [] # list of corresponding types of intent phrases
for intent, value in BOT_CONFIG['intents'].items():
    X_text += value['examples']
    y += [intent] * len(value['examples'])

# Based on TfidfVectorizer and LinearSVC program defines the best answer on user's question
vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(1,3), sublinear_tf=True)
X = vectorizer.fit_transform(X_text)
clf = LinearSVC()
clf.fit(X, y)

def get_intent(text):
    """
    This function returns intent on user's input.
    text: user's input
    """
    text_vector = vectorizer.transform([text]).toarray()[0]
    intent = clf.predict([text_vector])[0]
    
    # Processes posibility of user to enter word incorrectly (for example: 'Пирвет' instead of 'Привет' )
    for example in BOT_CONFIG['intents'][intent]['examples']:
        # distance — Calculated Levenshtein edit-distance between two strings. The edit distance is the number 
        # of characters that need to be substituted, inserted, or deleted, to transform s1 into s2.
        distance = nltk.edit_distance(text.lower(), example.lower())
        difference = distance / len(example)
        # 0.8 — the most affordable difference between enterd and real word
        if difference < 0.8:
            return intent

In [5]:
get_intent('Привіт')

'hello'

In [6]:
def response_by_intent(intent):
    """
    Returns randomized answer from list of responses of given intent
    """
    responses = BOT_CONFIG['intents'][intent]['responses']
    return random.choice(responses)

def get_failure_phrase():
    """
    Returns randomized answer from list of failure_phrases
    """
    failure_phrases = BOT_CONFIG['failure_phrases']
    return random.choice(failure_phrases)

In [7]:
# If intent is undefined, before using failure phrases to answer, Bot searchs response in dialogues.txt.
# dialogues.txt — big dataset of dialogues, formed using fiction literature 
# by Ilya Koziev (https://github.com/Koziev/NLP_Datasets/tree/master/Conversations/Data)

with open('dialogues.txt', encoding='Utf-8') as f:
    content = f.read()

def clean_str(replica, question=True):
    """
    replica: given phrase from dialogues.txt
    question: if given phrase is question — True, otherwise — False.
    Returns: cleaned phrase from inappropriate symbols
    """
    
    # variable alphabet — string with appropriate symbols
    if not question:
        # If given replica is not question then this replica will be an anwer for user's question 
        # and we add uppercase letters + punctuation signs.
        alphabet = ' 1234567890-йцукенгшщзхъфывапролджэёячсмитьбюqwertyuiopasdfghjklzxcvbnm' 
        alphabet = alphabet + alphabet.upper() + string.punctuation
        replica = replica[2:]
        replica = [c for c in replica if c in alphabet]
        return ''.join(replica)
    else:
        alphabet = ' 1234567890-йцукенгшщзхъфывапролджэёячсмитьбюqwertyuiopasdfghjklzxcvbnm' 
        replica = replica[2:].lower()
        replica = [c for c in replica if c in alphabet]
        return ''.join(replica)

    
# Takes data from 'dialogues.txt' preprocesses and forms dataset list with tuples [('Question', 'Answer'), ...]
dataset = []
blocks = content.split('\n\n')
for block in blocks:
    replicas = block.split('\n')[:2]
    if len(replicas) == 2:
        pair = (clean_str(replicas[0], question=True), clean_str(replicas[1], question=False))
        if pair[0] and pair[1]:
            dataset.append(pair)

dataset = list(set(dataset))

# Takes all words used in dataset and makes dict (key — word, 
# values — list of replicas' pairs (question, answer), which use corresponding word)
mega_dataset = {}  # word: [(Que, Answ), ...]  word: [(Que, Answ), ...] 
for question, answer in dataset:
    words = question.split(' ')
    for word in words:
        if word not in mega_dataset:
            mega_dataset[word] = []
        mega_dataset[word].append((question, answer))

In [8]:
def get_generative_replica(text):
    """
    text: user's input
    Returns answer on text from mega_dataset
    """
    # Prepare user's input for future analysis
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    user_words = text.split(' ')
    
    for word in user_words:
        if word in mega_dataset:
            for question, answer in mega_dataset[word]:
                # if length diference between user's input and question in mega_dataset is < 20% 
                # then consider this option
                if abs(len(text) - len(question)) / len(question) < 0.2:
                    distance = nltk.edit_distance(text, question)
                    difference = distance / len(question)
                    # 0.2 — the most affordable difference between enterd and real word
                    if difference < 0.2:
                        return answer

In [9]:
get_generative_replica('Привет')

'Как дела?'

In [10]:
# Dict for staticstical infofmation: counts how many times Bot used intent, 
# generative or stub (failure phrases) model to answer. 
stats = {'intent': 0, 'generative': 0, 'stub': 0}

In [11]:
def bot(text):
    """
    This function does main work on finding the most appropriate anwer on user's input
    text: user's input
    """
    # NLU
    intent = get_intent(text)
    
    # Answer generation
    if intent:
        stats['intent'] += 1
        return response_by_intent(intent)

    # generative model
    replica = get_generative_replica(text)
    if replica:
        stats['generative'] += 1
        return replica
    
    # return failure phrase
    stats['stub'] += 1
    return get_failure_phrase()

In [12]:
bot('Какой фильм посмотреть?')

'Однозначно, Матрица'

### Connect Telegram with Bot

In [13]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters


def start(update, context):
    """Send a message when the command /start is issued."""
    update.message.reply_text('Hi!')


def help_command(update, context):
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def ask_bot(update, context):
    response = bot(update.message.text)
    count = sum(stats.values())
    # print the Bot's stats
    print(f'{count}: intents {stats["intent"] / count:.2f}% '
          f'generative {stats["generative"] / count:.2f}% '
          f'stub {stats["stub"] / count:.2f}%', update.message.text, response)
    update.message.reply_text(response)


def main():
    """Start the bot."""
    updater = Updater("YOUR-TELEGRAM-BOT-KEY", use_context=True)
    dp = updater.dispatcher
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(CommandHandler("help", help_command))
    dp.add_handler(MessageHandler(Filters.text, ask_bot))
    updater.start_polling()
    updater.idle()

In [ ]:
main()